<a href="https://colab.research.google.com/github/hyunjin-lee-2002/Chapter01/blob/master/6_(colab)%E1%84%8B%E1%85%B5%E1%84%86%E1%85%B5%E1%84%8C%E1%85%B5_%E1%84%87%E1%85%AE%E1%86%AB%E1%84%85%E1%85%B2_%E1%84%86%E1%85%A9%E1%84%83%E1%85%A6%E1%86%AF_%E1%84%92%E1%85%A1%E1%86%A8%E1%84%89%E1%85%B3%E1%86%B8%E1%84%89%E1%85%B5%E1%84%8F%E1%85%B5%E1%84%80%E1%85%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🧠 PyTorch CNN 이미지 분류 모델 학습시키기

Fashion MNIST 이미지 분류 모델 학습 과정은 **데이터 준비**, **모델 정의**, **학습 설정**, 그리고 **학습 및 평가 루프**의 네 단계로 요약됩니다.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
from torchvision.transforms import v2
from torch.utils.data import DataLoader

## 1. 데이터 준비 및 분할

모델 학습을 위해 기존의 Fashion MNIST 데이터를 학습, 검증, 테스트 세트로 나누고 전처리를 적용합니다.

### 데이터셋 분할

* **`random_split`**:

    - 초기 **테스트 데이터셋(10,000개)** 을 `random_split` 함수를 사용하여 **검증 데이터셋 (5,000개)** 과 **최종 테스트 데이터셋 (5,000개)** 으로 분할했습니다.

* **Transform 적용**:

    * **학습 데이터 (`transforms_train`)** :

        - **이미지 증강**(`RandomHorizontalFlip`, `RandomResizedCrop`, `RandomRotation`)과 **전처리**(`ToImage`, `ToDtype(0~1 스케일링)`, `Normalize`)를 모두 적용했습니다.

    * **검증/테스트 데이터 (`transforms_test`)**

        - **이미지 증강을 제외**하고 **전처리**만 적용했습니다.
    
* **`DataLoader`**:

    - 배치 크기 128 (Train, `shuffle=True`, `drop_last=True`)과 32 (Validation/Test)로 설정하여 데이터를 모델에 공급합니다.


In [2]:
from torch.utils.data import random_split

# =============================================================================
# 데이터 준비 (전처리, 증강, 데이터로더)
# =============================================================================

# 1-1. 전처리 및 데이터 증강 정의
# 학습 데이터에는 이미지 증강(Augmentation)을 포함하여 모델의 일반화 성능을 높입니다.
transforms_train = v2.Compose(
    [
        v2.ToImage(),                          # PIL Image를 PyTorch 텐서로 변환
        v2.RandomHorizontalFlip(),             # 무작위 좌우 반전 (증강)
        v2.RandomResizedCrop(size=28),         # 무작위로 잘라서 28x28로 리사이즈 (증강)
        v2.RandomRotation(degrees=10),         # 무작위 회전 (증강)
        v2.ToDtype(dtype=torch.float32, scale=True), # 픽셀 값을 0~1 범위의 float 타입으로 스케일링
        v2.Normalize(mean=[0.286], std=[0.353]), # 학습 데이터의 평균/표준편차를 이용해 표준화 (Normalize)
    ]
)

# 테스트/검증 데이터에는 증강을 제외하고 필수적인 전처리만 적용합니다.
transforms_test = v2.Compose(
    [
        v2.ToImage(),                          # PIL Image를 PyTorch 텐서로 변환
        v2.ToDtype(dtype=torch.float32, scale=True), # 픽셀 값을 0~1 범위의 float 타입으로 스케일링
        v2.Normalize(mean=[0.286], std=[0.353]), # 표준화
    ]
)

# 1-2. FashionMNIST 데이터셋 로드
train_dataset = datasets.FashionMNIST(
    root='./fashion_mnist',
    train=True,
    download=True,
    transform=transforms_train,  # 학습 데이터에 증강/전처리 적용
)

test_dataset_full = datasets.FashionMNIST(
    root='./fashion_mnist',
    train=False,
    download=True,
    transform=transforms_test,   # 테스트 데이터에는 전처리만 적용
)

# 1-3. 테스트 데이터셋을 검증(Validation)과 최종 테스트(Test)로 분할
# 만 개의 테스트 데이터 중 5000개를 검증용으로, 나머지 5000개를 최종 테스트용으로 분할
val_dataset, test_dataset = random_split(test_dataset_full, [5000, 5000])

# 1-4. DataLoader 객체 생성
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=128,
    shuffle=True,          # 학습 시 데이터 순서를 섞음
    drop_last=True         # 마지막 배치를 버려 배치 크기를 일정하게 유지
)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=32)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=32)

100%|██████████| 26.4M/26.4M [00:01<00:00, 13.4MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 211kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.95MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 11.0MB/s]



## 2. 모델 및 학습 환경 설정

### 모델 구조 (`CNNModel`)

* **CNN 블록**: 두 개의 `Conv2d` 레이어와 `MaxPool2d` 레이어를 조합했습니다.

    * `Conv2d(1, 32, ...)`와 `Conv2d(32, 64, ...)`로 채널 수를 증가시켰습니다.

    * `MaxPool2d(2, 2)`를 두 번 적용하여 특징 맵의 크기를 $28 \times 28 \to 14 \times 14 \to 7 \times 7$로 축소했습니다.

* **Fully Connected**: Flatten 직후 벡터 크기인 **3136**을 첫 번째 `nn.Linear` 레이어의 입력 차원으로 정확히 설정했습니다.

* **Dropout**: `nn.Dropout(0.2)`를 적용하여 오버피팅을 방지했습니다.

### 설정

* **Loss Function**: 10개 클래스 분류 문제이므로 `nn.CrossEntropyLoss()`를 사용했습니다.

* **Optimizer**: **`optim.Adam`** 을 사용하여 모델 파라미터를 업데이트합니다.

* **Device**: GPU(`cuda`) 사용 가능 여부에 따라 디바이스를 설정했습니다.


In [ ]:
# =============================================================================
# CNN 모델 정의
# =============================================================================

class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Conv1: 1채널(Grayscale) 입력 -> 32채널 출력. 커널 3x3, 패딩 1로 크기 유지 (28x28 -> 28x28)
        self.conv1 = nn.Conv2d(1, 32, 3, 1, 1)
        # Conv2: 32채널 입력 -> 64채널 출력. 커널 3x3, 패딩 1로 크기 유지 (14x14 -> 14x14)
        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        # MaxPool: 2x2 풀링, 스트라이드 2 (높이/너비 1/2 축소)
        self.maxpool = nn.MaxPool2d(2, 2)

        # 학습 시 20% 확률로 뉴런 비활성화 (오버피팅 방지)
        self.dropout = nn.Dropout(0.2)

        # 다차원 피처맵을 1차원 벡터로 펼침 (Linear 레이어 입력을 위해 필요)
        self.flatten = nn.Flatten()

        # Linear1 입력 차원: 64 (채널) * 7 * 7 (최종 피처맵 크기) = 3136
        self.linear1 = nn.Linear(3136, 64)
        # Linear2 출력 차원: 10 (클래스 개수)
        self.linear2 = nn.Linear(64, 10)

    def forward(self, x):
        # 첫 번째 Conv-ReLU-Pool 블록
        x = F.relu(self.conv1(x)) # Conv2d 후 활성화 함수 (함수형 F.relu 사용)
        x = self.maxpool(x)       # Max Pooling (28x28 -> 14x14)

        # 두 번째 Conv-ReLU-Pool 블록
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)       # Max Pooling (14x14 -> 7x7)

        # Dropout 적용 후 벡터로 펼치기
        x = self.dropout(x)
        x = self.flatten(x)       # (N, 64, 7, 7) -> (N, 3136)

        # 완전 연결 레이어
        x = F.relu(self.linear1(x))
        output = self.linear2(x)

        return output

In [ ]:
# =============================================================================
# 모델, 손실 함수, 옵티마이저 설정
# =============================================================================

model = CNNModel()

# GPU 사용 가능 여부 확인 및 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

CNNModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=3136, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=10, bias=True)
)

In [ ]:
# 손실 함수: 다중 분류 문제에 적합한 CrossEntropyLoss 사용
loss_fn = nn.CrossEntropyLoss()
# 옵티마이저: Adam 사용
optimizer = optim.Adam(model.parameters())

In [ ]:
# =============================================================================
# 학습 루프 실행
# =============================================================================

epochs = 5
step = 0
print(f"Training started on device: {device}")

for epoch in range(epochs):
    # 모델을 학습 모드로 전환
    model.train()
    for train_batch in train_dataloader:
        inputs = train_batch[0].to(device)
        labels = train_batch[1].to(device)

        # 1. 순전파 (Forward Pass)
        preds = model(inputs)
        loss = loss_fn(preds, labels)

        # 2. 역전파 (Backward Pass)
        loss.backward()            # 그래디언트 계산
        optimizer.step()           # 파라미터 업데이트
        optimizer.zero_grad()      # 그래디언트 초기화

        step += 1
        if step % 100 == 0:
            print(f'step {step}, train loss: {loss.item():.4f}')

    # 에폭 종료 후 검증 데이터로 모델 평가
    val_loss, val_acc = evaluate(val_dataloader, model, loss_fn)
    print(f'epoch {epoch+1}/{epochs}, val loss: {val_loss:.4f}, val acc: {val_acc:.4f}\n')

print('Training finished!')


## 3. 평가 함수 (`evaluate`)

학습 과정에서 모델의 성능을 측정하기 위한 함수입니다.

* **모드 전환**: **`model.eval()`** 을 호출하여 Dropout과 같은 레이어를 비활성화합니다.

* **그래디언트 비활성화**: **`with torch.no_grad():`** 블록 내에서 실행하여 메모리 사용을 줄이고 속도를 높입니다.

* **정확도 계산**:

    1.  `preds` 텐서에 `torch.argmax(dim=1)`을 적용하여 모델이 예측한 클래스 인덱스를 구합니다.

    2.  예측 레이블과 실제 레이블이 일치하는 개수(`(pred_labels == labels).sum()`)를 누적하여 정확도(Accuracy)를 계산합니다.


In [ ]:
# =============================================================================
# 평가 함수 정의
# =============================================================================

def evaluate(dataloader, model, loss_fn):
    # 모델을 평가 모드로 전환 (Dropout, BatchNorm 등이 비활성화됨)
    model.eval()
    losses = []
    correct = 0
    total = 0

    # 평가 시에는 그래디언트 계산이 필요 없으므로 기록을 끔 (메모리 절약)
    with torch.no_grad():
        for batch in dataloader:
            inputs = batch[0].to(device)
            labels = batch[1].to(device)

            # 1. 예측값 계산
            preds = model(inputs)  # [배치 크기, 클래스 개수]

            # 2. Loss 계산
            loss = loss_fn(preds, labels)
            losses.append(loss.item())

            # 3. 정확도 계산
            # torch.argmax(dim=1)로 가장 높은 값의 인덱스(예측 클래스)를 추출
            pred_labels = torch.argmax(preds, dim=1)
            # 정답 개수 누적 (True: 1, False: 0으로 간주하여 합산)
            correct += (pred_labels == labels).sum().item()
            total += len(preds)

    # 평균 Loss와 정확도 계산
    avg_loss = sum(losses) / len(losses)
    acc = correct / total

    return avg_loss, acc


## 4. 학습 루프 및 최종 평가

각 에폭마다 학습을 진행하고 검증 데이터로 성능을 확인합니다.

1.  **모델 학습 (`model.train()`)**:

    * 데이터를 모델에 입력하여 Loss를 계산합니다.

    * `loss.backward()`로 그래디언트를 계산하고, `optimizer.step()`으로 파라미터를 업데이트합니다.

    * `optimizer.zero_grad()`로 그래디언트를 초기화합니다.

2.  **검증 (`val_dataloader`)**:

    * 매 에폭 학습이 끝날 때, `evaluate(val_dataloader, ...)`를 호출하여 **검증 손실 (`val loss`)** 과 **검증 정확도 (`val acc`)** 를 출력하고, 모델 성능 개선 여부를 모니터링합니다.

3.  **최종 테스트**:

    * 전체 학습이 완료된 후, `evaluate(test_dataloader, ...)`를 호출하여 **최종 테스트 데이터셋** 에 대한 Loss와 Accuracy를 계산합니다.

In [ ]:

# =============================================================================
# 최종 테스트 데이터로 평가
# =============================================================================

test_loss, test_acc = evaluate(test_dataloader, model, loss_fn)
print(f"\n[Final Test Result] Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

- [Colab 노트북](https://colab.research.google.com/drive/1_r06QY2QhpMt_Gexafgk3qJggiG2Iap8#scrollTo=x-aa234X5ajk)에서 함께 아래 코드를 실행해보기

    - 링크 접속 > 파일 > "Drive에 사본 저장" 클릭 $\to$ 편집 권한이 생김

    - 런타임 > 런타임 유형 변경 > T4 GPU 선택하기

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import v2

# =============================================================================
# 1. 데이터 준비 (전처리, 증강, 데이터로더)
# =============================================================================

# 1-1. 전처리 및 데이터 증강 정의
# 학습 데이터에는 이미지 증강(Augmentation)을 포함하여 모델의 일반화 성능을 높입니다.
transforms_train = v2.Compose(
    [
        v2.ToImage(),                          # PIL Image를 PyTorch 텐서로 변환
        v2.RandomHorizontalFlip(),             # 무작위 좌우 반전 (증강)
        v2.RandomResizedCrop(size=28),         # 무작위로 잘라서 28x28로 리사이즈 (증강)
        v2.RandomRotation(degrees=10),         # 무작위 회전 (증강)
        v2.ToDtype(dtype=torch.float32, scale=True), # 픽셀 값을 0~1 범위의 float 타입으로 스케일링
        v2.Normalize(mean=[0.286], std=[0.353]), # 학습 데이터의 평균/표준편차를 이용해 표준화 (Normalize)
    ]
)

# 테스트/검증 데이터에는 증강을 제외하고 필수적인 전처리만 적용합니다.
transforms_test = v2.Compose(
    [
        v2.ToImage(),                          # PIL Image를 PyTorch 텐서로 변환
        v2.ToDtype(dtype=torch.float32, scale=True), # 픽셀 값을 0~1 범위의 float 타입으로 스케일링
        v2.Normalize(mean=[0.286], std=[0.353]), # 표준화
    ]
)

# 1-2. FashionMNIST 데이터셋 로드
train_dataset = datasets.FashionMNIST(
    root='./fashion_mnist',
    train=True,
    download=True,
    transform=transforms_train,  # 학습 데이터에 증강/전처리 적용
)

test_dataset_full = datasets.FashionMNIST(
    root='./fashion_mnist',
    train=False,
    download=True,
    transform=transforms_test,   # 테스트 데이터에는 전처리만 적용
)

# 1-3. 테스트 데이터셋을 검증(Validation)과 최종 테스트(Test)로 분할
# 만 개의 테스트 데이터 중 5000개를 검증용으로, 나머지 5000개를 최종 테스트용으로 분할
val_dataset, test_dataset = random_split(test_dataset_full, [5000, 5000])

# 1-4. DataLoader 객체 생성
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=128,
    shuffle=True,          # 학습 시 데이터 순서를 섞음
    drop_last=True         # 마지막 배치를 버려 배치 크기를 일정하게 유지
)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=32)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=32)


# =============================================================================
# 2. CNN 모델 정의
# =============================================================================

class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Conv1: 1채널(Grayscale) 입력 -> 32채널 출력. 커널 3x3, 패딩 1로 크기 유지 (28x28 -> 28x28)
        self.conv1 = nn.Conv2d(1, 32, 3, 1, 1)
        # Conv2: 32채널 입력 -> 64채널 출력. 커널 3x3, 패딩 1로 크기 유지 (14x14 -> 14x14)
        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        # MaxPool: 2x2 풀링, 스트라이드 2 (높이/너비 1/2 축소)
        self.maxpool = nn.MaxPool2d(2, 2)

        # 학습 시 20% 확률로 뉴런 비활성화 (오버피팅 방지)
        self.dropout = nn.Dropout(0.2)

        # 다차원 피처맵을 1차원 벡터로 펼침 (Linear 레이어 입력을 위해 필요)
        self.flatten = nn.Flatten()

        # Linear1 입력 차원: 64 (채널) * 7 * 7 (최종 피처맵 크기) = 3136
        self.linear1 = nn.Linear(3136, 64)
        # Linear2 출력 차원: 10 (클래스 개수)
        self.linear2 = nn.Linear(64, 10)

    def forward(self, x):
        # 첫 번째 Conv-ReLU-Pool 블록
        x = F.relu(self.conv1(x)) # Conv2d 후 활성화 함수 (함수형 F.relu 사용)
        x = self.maxpool(x)       # Max Pooling (28x28 -> 14x14)

        # 두 번째 Conv-ReLU-Pool 블록
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)       # Max Pooling (14x14 -> 7x7)

        # Dropout 적용 후 벡터로 펼치기
        x = self.dropout(x)
        x = self.flatten(x)       # (N, 64, 7, 7) -> (N, 3136)

        # 완전 연결 레이어
        x = F.relu(self.linear1(x))
        output = self.linear2(x)

        return output


# =============================================================================
# 3. 모델, 손실 함수, 옵티마이저 설정
# =============================================================================

model = CNNModel()

# GPU 사용 가능 여부 확인 및 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 손실 함수: 다중 분류 문제에 적합한 CrossEntropyLoss 사용
loss_fn = nn.CrossEntropyLoss()
# 옵티마이저: Adam 사용
optimizer = optim.Adam(model.parameters())


# =============================================================================
# 4. 평가 함수 정의
# =============================================================================

def evaluate(dataloader, model, loss_fn):
    # 모델을 평가 모드로 전환 (Dropout, BatchNorm 등이 비활성화됨)
    model.eval()
    losses = []
    correct = 0
    total = 0

    # 평가 시에는 그래디언트 계산이 필요 없으므로 기록을 끔 (메모리 절약)
    with torch.no_grad():
        for batch in dataloader:
            inputs = batch[0].to(device)
            labels = batch[1].to(device)

            # 1. 예측값 계산
            preds = model(inputs)  # [배치 크기, 클래스 개수]

            # 2. Loss 계산
            loss = loss_fn(preds, labels)
            losses.append(loss.item())

            # 3. 정확도 계산
            # torch.argmax(dim=1)로 가장 높은 값의 인덱스(예측 클래스)를 추출
            pred_labels = torch.argmax(preds, dim=1)
            # 정답 개수 누적 (True: 1, False: 0으로 간주하여 합산)
            correct += (pred_labels == labels).sum().item()
            total += len(preds)

    # 평균 Loss와 정확도 계산
    avg_loss = sum(losses) / len(losses)
    acc = correct / total

    return avg_loss, acc


# =============================================================================
# 5. 학습 루프 실행
# =============================================================================

epochs = 5
step = 0
print(f"Training started on device: {device}")

for epoch in range(epochs):
    # 모델을 학습 모드로 전환
    model.train()
    for train_batch in train_dataloader:
        inputs = train_batch[0].to(device)
        labels = train_batch[1].to(device)

        # 1. 순전파 (Forward Pass)
        preds = model(inputs)
        loss = loss_fn(preds, labels)

        # 2. 역전파 (Backward Pass)
        loss.backward()            # 그래디언트 계산
        optimizer.step()           # 파라미터 업데이트
        optimizer.zero_grad()      # 그래디언트 초기화

        step += 1
        if step % 100 == 0:
            print(f'step {step}, train loss: {loss.item():.4f}')

    # 에폭 종료 후 검증 데이터로 모델 평가
    val_loss, val_acc = evaluate(val_dataloader, model, loss_fn)
    print(f'epoch {epoch+1}/{epochs}, val loss: {val_loss:.4f}, val acc: {val_acc:.4f}\n')

print('Training finished!')

# =============================================================================
# 6. 최종 테스트 데이터로 평가
# =============================================================================

test_loss, test_acc = evaluate(test_dataloader, model, loss_fn)
print(f"\n[Final Test Result] Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")